##  **Creating the credential to access the Storage Account**

The credential name is the URL to the storage account. This will stablish the link between the credential and the XE Target

Ref: [Creating Credentials](https://docs.microsoft.com/en-us/sql/t-sql/statements/create-database-scoped-credential-transact-sql?view=sql-server-ver15&WT.mc_id=DP-MVP-4014132)

In [18]:
create master key encryption by password='6964xpahw@@'
GO

CREATE DATABASE SCOPED CREDENTIAL [https://sqlxe.blob.core.windows.net/xelogfiles/LoadingLogs]
WITH IDENTITY='SHARED ACCESS SIGNATURE'
, SECRET = 'sv=2019-12-12&ss=bfqt&srt=sco&sp=rwdlacupx&se=2021-09-05T23:51:34Z&st=2020-11-04T16:51:34Z&spr=https&sig=lnN1um%2FEt2iJwHVLa%2FtvLgig38dBYHKbAtmFsdQMF8Q%3D'
GO

CREATE DATABASE SCOPED CREDENTIAL [https://sqlxe.blob.core.windows.net/xelogfiles]
WITH IDENTITY='SHARED ACCESS SIGNATURE'
, SECRET = 'sv=2019-12-12&ss=bfqt&srt=sco&sp=rwdlacupx&se=2021-09-05T23:51:34Z&st=2020-11-04T16:51:34Z&spr=https&sig=lnN1um%2FEt2iJwHVLa%2FtvLgig38dBYHKbAtmFsdQMF8Q%3D'
GO

: Msg 15578, Level 16, State 1, Line 1
There is already a master key in the database. Please drop it before performing this statement.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.029

In [17]:
drop event session queries

: Msg 102, Level 15, State 1, Line 1
Incorrect syntax near 'queries'.

Total execution time: 00:00:00.007

##  **Creating Extended Events Session**

Ref: [Extended Events With Azure SQL](https://docs.microsoft.com/en-us/azure/azure-sql/database/xevent-db-diff-from-svr?WT.mc\_id=DP-MVP-4014132)

In [20]:
CREATE EVENT SESSION [Queries] ON DATABASE 
ADD EVENT sqlserver.rpc_completed(SET collect_statement=(1)
    ACTION(sqlserver.database_name)),
ADD EVENT sqlserver.rpc_starting(
    ACTION(sqlserver.database_name)),
ADD EVENT sqlserver.sql_statement_completed(
    ACTION(sqlserver.database_name)),
ADD EVENT sqlserver.sql_statement_starting(
    ACTION(sqlserver.database_name))
ADD TARGET package0.event_file(SET filename=N'https://sqlxe.blob.core.windows.net/xelogfiles/LoadingLogs/Logs.xel',max_file_size=(10))
WITH (MAX_MEMORY=4096 KB,EVENT_RETENTION_MODE=ALLOW_SINGLE_EVENT_LOSS,MAX_DISPATCH_LATENCY=30 SECONDS,MAX_EVENT_SIZE=0 KB,MEMORY_PARTITION_MODE=NONE,TRACK_CAUSALITY=OFF,STARTUP_STATE=ON)
GO

Alter Event Session [Queries] on database state=start
go

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.104

##  **Querying the Extended Events**

On this first example, if made on the planned order, this query will show a direct count executed over the table, illustrating a **pushdown**

In [ ]:
with qry as 
(select 
theNodes.event_data.value('(@name)[1]','nvarchar(max)') as [EventName],
theNodes.event_data.value('(@timestamp)[1]','datetime') as [Date],
theNodes.event_data.value('(action[@name="database_name"]/value)[1]','nvarchar(max)') as database_name,
theNodes.event_data.value('(data[@name="cpu_time"]/value)[1]','bigint') as cpu,
theNodes.event_data.value('(data[@name="duration"]/value)[1]','bigint') as duration,
theNodes.event_data.value('(data[@name="physical_reads"]/value)[1]','bigint') as reads,
theNodes.event_data.value('(data[@name="writes"]/value)[1]','bigint') as writes,
theNodes.event_data.value('(data[@name="statement"]/value)[1]','nvarchar(max)') as statement,
theNodes.event_data.value('(data[@name="batch_text"]/value)[1]','nvarchar(max)') as batch_text
from
(SELECT         
        CAST(event_data AS XML) AS [event_data]  -- TODO: In ssms.exe results grid, double-click this cell!
    FROM
        sys.fn_xe_file_target_read_file
            (
                -- TODO: Fill in Storage Account name, and the associated Container name.
                -- TODO: The name of the .xel file needs to be an exact match to the files in the storage account Container (You can use Storage Account explorer from the portal to find out the exact file names or you can retrieve the name using the following DMV-query: select target_data from sys.dm_xe_database_session_targets. The 3rd xml-node, "File name", contains the name of the file currently written to.)
                'https://sqlxe.blob.core.windows.net/xelogfiles/LoadingLogs/Logs_0_132489132692850000.xel',
                null,null , null
            )) as theData
 cross apply theData.event_data.nodes('//event') theNodes(event_data) )
 select * from qry
 where statement like '%FactInternetSales%' or batch_text like '%FactInternetSales%'
 order by [Date] desc


## **2nd Extended Events Check**

This time the count was broke down by partitions. Polybase manage to make the pushdown by partitions if the destination table is partitioned.

In [ ]:
with qry as 
(select 
theNodes.event_data.value('(@name)[1]','nvarchar(max)') as [EventName],
theNodes.event_data.value('(@timestamp)[1]','datetime') as [Date],
theNodes.event_data.value('(action[@name="database_name"]/value)[1]','nvarchar(max)') as database_name,
theNodes.event_data.value('(data[@name="cpu_time"]/value)[1]','bigint') as cpu,
theNodes.event_data.value('(data[@name="duration"]/value)[1]','bigint') as duration,
theNodes.event_data.value('(data[@name="physical_reads"]/value)[1]','bigint') as reads,
theNodes.event_data.value('(data[@name="writes"]/value)[1]','bigint') as writes,
theNodes.event_data.value('(data[@name="statement"]/value)[1]','nvarchar(max)') as statement,
theNodes.event_data.value('(data[@name="batch_text"]/value)[1]','nvarchar(max)') as batch_text
from
(SELECT         
        CAST(event_data AS XML) AS [event_data]  -- TODO: In ssms.exe results grid, double-click this cell!
    FROM
        sys.fn_xe_file_target_read_file
            (
                -- TODO: Fill in Storage Account name, and the associated Container name.
                -- TODO: The name of the .xel file needs to be an exact match to the files in the storage account Container (You can use Storage Account explorer from the portal to find out the exact file names or you can retrieve the name using the following DMV-query: select target_data from sys.dm_xe_database_session_targets. The 3rd xml-node, "File name", contains the name of the file currently written to.)
                'https://sqlxe.blob.core.windows.net/xelogfiles/LoadingLogs/Logs_0_132489132692850000.xel',
                null,null , null
            )) as theData
 cross apply theData.event_data.nodes('//event') theNodes(event_data) )
 select * from qry
 where statement like '%FactProductInventory%' or batch_text like '%FactProductInventory%'
 order by [Date] desc